# **1- Reading Data**

In [20]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import regex as re,sys
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Ignore Warnings
import warnings
#warnings.filterwarnings("ignore")
#warnings.warn("this will not show")

In [5]:
# to install pandas-profiling for show the EDA
#pip install pandas-profiling

In [6]:
#from google.colab import drive
#drive.mount('/content/drive') 

In [7]:
import pandas as pd
#data = pd.read_excel("/content/drive/MyDrive/DeGatto Project/Output Womens Clothing E-Commerce Reviews.xlsx")
data = pd.read_excel("Output Womens Clothing E-Commerce Reviews.xlsx")

# **2- Exploratory Data Analysis**

In [8]:
# Checking the data information
data.info()
"""
As it can be seen, we have 13 columns and 23467 entries.
From these columns, 7 of them are 
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23467 entries, 0 to 23466
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       23467 non-null  int64 
 1   Clothing ID      23467 non-null  int64 
 2   Review Text      22641 non-null  object
 3   Rating           23467 non-null  int64 
 4   Recommended IND  23467 non-null  int64 
 5   Category name    23467 non-null  object
 6   Sentiment        23467 non-null  object
 7   Size             3485 non-null   object
 8   Material         2703 non-null   object
 9   Longevity        57 non-null     object
 10  Color            2933 non-null   object
 11  General          302 non-null    object
 12  Comfort          561 non-null    object
dtypes: int64(4), object(9)
memory usage: 2.3+ MB


'\nAs it can be seen, we have 13 columns and 23467 entries.\nFrom these columns, 7 of them are \n'

In [16]:
data.head()

,Unnamed: 0,Clothing ID,Review Text,Rating,Recommended IND,Category name,Sentiment,Size,Material,Longevity,Color,General,Comfort
0,96,861,i am in need of easy comfortable tops for ever...,3,0,Knits,Neutral,POS,POS,NaN,POS,NaN,POS
1,123,1133,i read the previous reviews and had hoped that...,4,1,Jackets,Positive,POS,POS,NaN,POS,NaN,POS
2,240,872,this is exactly what i was expecting cute comf...,5,1,Knits,Positive,NaN,POS,NaN,POS,NaN,POS
3,276,1104,this dress is gorgeous i love it i bought it t...,5,1,Dresses,Positive,NaN,POS,NaN,POS,NaN,POS
4,310,836,love this top made with 100 cotton a vintage l...,5,1,Tops,Positive,POS,NaN,NaN,POS,NaN,POS




1) Clothing ID: Integer Categorical variable that refers to the specific piece being reviewed.

2) Review Text: String variable for the review body.

3) Rating: Positive Ordinal Integer variable for the product score granted by the customer from 1 Worst, to 5 Best.

4) Recommended IND: Binary variable stating where the customer recommends the product where 1 is recommended, 0 is not recommended.

5) Category Name: Categorical name of the product department name.

6) Sentiment:


In [9]:
#data.profile_report()

In [10]:
def first_looking(df):
  print(data.info())
  print(data.describe())
  print(data.isnull().sum()/len(data)*100)
  print(data[data['Review Text'].isnull()])
  missing_values=data.isnull().sum()
  print(missing_values.sort_values(ascending=False))

In [11]:
first_looking(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23467 entries, 0 to 23466
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       23467 non-null  int64 
 1   Clothing ID      23467 non-null  int64 
 2   Review Text      22641 non-null  object
 3   Rating           23467 non-null  int64 
 4   Recommended IND  23467 non-null  int64 
 5   Category name    23467 non-null  object
 6   Sentiment        23467 non-null  object
 7   Size             3485 non-null   object
 8   Material         2703 non-null   object
 9   Longevity        57 non-null     object
 10  Color            2933 non-null   object
 11  General          302 non-null    object
 12  Comfort          561 non-null    object
dtypes: int64(4), object(9)
memory usage: 2.3+ MB
None
         Unnamed: 0   Clothing ID        Rating  Recommended IND
count  23467.000000  23467.000000  23467.000000     23467.000000
mean   11742.862062    918.1089

# **3- PRE-PROCESSING**


    OverSampling & UnderSampling
    Remove Punctuation
    Lowercase
    Tokenization
    Stopwords (We will try both ways remove and let them stay)
    Lemmatization / Stemming
    Remove numbers
    Remove pronouns (Check if needed to remove)
    Remove consecutive repeating words (to see if it should be done)
    Vectorization
    TF-IDF



In [18]:
data = data.drop(['Unnamed: 0', 'Clothing ID'], axis=1)
data.head()

,Review Text,Rating,Recommended IND,Category name,Sentiment,Size,Material,Longevity,Color,General,Comfort
0,i am in need of easy comfortable tops for ever...,3,0,Knits,Neutral,POS,POS,NaN,POS,NaN,POS
1,i read the previous reviews and had hoped that...,4,1,Jackets,Positive,POS,POS,NaN,POS,NaN,POS
2,this is exactly what i was expecting cute comf...,5,1,Knits,Positive,NaN,POS,NaN,POS,NaN,POS
3,this dress is gorgeous i love it i bought it t...,5,1,Dresses,Positive,NaN,POS,NaN,POS,NaN,POS
4,love this top made with 100 cotton a vintage l...,5,1,Tops,Positive,POS,NaN,NaN,POS,NaN,POS


In [21]:
# Removing Punctuations and Numbers from the Text
def remove_punctuations_numbers(inputs):
    return re.sub(r'[^a-zA-Z]', ' ', inputs)


data['Review Text'] = data['Review Text'].apply(remove_punctuations_numbers)

TypeError: expected string or buffer

In [31]:
def tokenization(inputs):
    return word_tokenize(inputs)


data['text_tokenized'] = data['Review Text'].apply(tokenization)
data['text_tokenized'].head()



TypeError: expected string or bytes-like object

# **4- MODELS**

We will directly work on ACCURACY, RECALL, PRECISION, F1-SCORE and take one of them NOT NECESSIRALY accuracy, depending on the best fitted, and the sampling of the data

    Naive Bayes
    Logistic Regression
    SVM (different ones, polynomial, linear, SVC, ...)
    LSTM

